<a href="https://colab.research.google.com/github/FedorTaggenbrock/data_intensive_systems/blob/main/Spark_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=88254e5de68c20f44c87133595efab593f1092a6a36cb8facceed798b13145d4
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
import pyspark
import math
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark import RDD
from pyspark import SparkContext

In [ ]:
spark = SparkSession.builder.appName("Practise").getOrCreate()

In [ ]:
def kMeans(data: RDD, k: int, maxIterations: int) -> list:
    # Initialize centroids randomly
    centroids = [tuple(x) for x in data.takeSample(withReplacement=False, num=k)]

    # Define a custom distance function
    def distance(point1, point2):
        return math.sqrt(sum([(x - y)**2 for x, y in zip(point1, point2)]))

    # Iterate until convergence or until the maximum number of iterations is reached
    for i in range(maxIterations):
        # Assign each point to the closest centroid
        clusters = data.map(lambda point: (min(centroids, key=lambda centroid: distance(point, centroid)), point)).groupByKey()

        # Compute new centroids as the mean of the points in each cluster
        newCentroids = clusters.mapValues(lambda points: tuple([sum(x) / len(points) for x in zip(*points)])).collect()

        # Update centroids
        for oldCentroid, newCentroid in newCentroids:
            index = centroids.index(oldCentroid)
            centroids[index] = newCentroid

    return [list(x) for x in centroids]


This code defines a kMeans function that takes an RDD of data points (represented as lists of floats), the number of clusters k, and the maximum number of iterations as input. The function initializes the centroids randomly and then iterates until convergence or until the maximum number of iterations is reached. In each iteration, each point is assigned to the closest centroid using a custom distance function (in this case, the Euclidean distance), and then new centroids are computed as the mean of the points in each cluster. Finally, the function returns the computed centroids.

This is just one example of how you could implement K-Means clustering with a custom distance function in PySpark using map and reduce functions. You could adapt this code to use a different distance function or to implement other variations of the K-Means algorithm.

In [ ]:
# Create an RDD of data points
data = spark.sparkContext.parallelize([
    [0.0, 0.0],
    [1.0, 1.0],
    [9.0, 8.0],
    [8.0, 9.0]
])

# Cluster the data into two clusters
centroids = kMeans(data, k=2, maxIterations=10)

# Print the resulting centroids
for centroid in centroids:
    print(centroid)

[0.5, 0.5]
[8.5, 8.5]


In [ ]:
print("test")